# LangChain 구조화된 출력

구조화된 출력을 사용하면 에이전트가 특정하고 예측 가능한 형식으로 데이터를 반환할 수 있습니다. 자연어 응답을 구문 분석하는 대신 애플리케이션에서 직접 사용할 수 있는 JSON 객체, Pydantic 모델 또는 데이터클래스 형태로 구조화된 데이터를 얻을 수 있습니다.

LangChain의 `create_agent`는 구조화된 출력을 자동으로 처리합니다. 사용자가 원하는 구조화된 출력 스키마를 설정하면 모델이 구조화된 데이터를 생성할 때 캡처, 검증되어 에이전트 상태의 `'structured_response'` 키에 반환됩니다.

## 사전 준비

환경 변수를 설정합니다.

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

## Response Format

에이전트가 구조화된 데이터를 반환하는 방법을 제어합니다:

- **`ToolStrategy[StructuredResponseT]`**: 구조화된 출력에 도구 호출 사용
- **`ProviderStrategy[StructuredResponseT]`**: 제공자 네이티브 구조화된 출력 사용
- **`type[StructuredResponseT]`**: 스키마 유형 - 모델 기능에 따라 최적의 전략을 자동으로 선택
- **`None`**: 구조화된 출력 없음

스키마 유형이 직접 제공되면 LangChain이 자동으로 선택합니다:
- 네이티브 구조화된 출력을 지원하는 모델(예: OpenAI, Grok)에는 `ProviderStrategy`
- 다른 모든 모델에는 `ToolStrategy`

구조화된 응답은 에이전트의 최종 상태의 `structured_response` 키에 반환됩니다.

## Provider Strategy

일부 모델 제공자(현재 OpenAI 및 Grok만 해당)는 API를 통해 구조화된 출력을 네이티브로 지원합니다. 사용 가능한 경우 가장 신뢰할 수 있는 방법입니다.

스키마 유형을 `create_agent.response_format`에 직접 전달하면 LangChain이 자동으로 `ProviderStrategy`를 사용합니다.

### Pydantic 모델

In [ ]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

class ContactInfo(BaseModel):
    """Contact information for a person."""
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address of the person")
    phone: str = Field(description="The phone number of the person")

# 모델 및 에이전트 생성
model = ChatOpenAI(model="gpt-4.1-mini")
agent = create_agent(
    model=model,
    tools=[],
    response_format=ContactInfo  # ProviderStrategy 자동 선택
)

# 에이전트 실행
result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

# 구조화된 응답 확인
print(result["structured_response"])
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

### 데이터클래스

In [ ]:
from dataclasses import dataclass
from langchain.agents import create_agent

@dataclass
class ContactInfo:
    """Contact information for a person."""
    name: str        # The name of the person
    email: str       # The email address of the person
    phone: str       # The phone number of the person

agent = create_agent(
    model=model,
    tools=[],
    response_format=ContactInfo  # ProviderStrategy 자동 선택
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: Jane Smith, jane@example.com, (555) 987-6543"}]
})

print(result["structured_response"])
# ContactInfo(name='Jane Smith', email='jane@example.com', phone='(555) 987-6543')

### TypedDict

In [ ]:
from typing_extensions import TypedDict
from langchain.agents import create_agent

class ContactInfo(TypedDict):
    """Contact information for a person."""
    name: str        # The name of the person
    email: str       # The email address of the person
    phone: str       # The phone number of the person

agent = create_agent(
    model=model,
    tools=[],
    response_format=ContactInfo  # ProviderStrategy 자동 선택
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: Bob Johnson, bob@example.com, (555) 111-2222"}]
})

print(result["structured_response"])
# {'name': 'Bob Johnson', 'email': 'bob@example.com', 'phone': '(555) 111-2222'}

## Tool Calling Strategy

네이티브 구조화된 출력을 지원하지 않는 모델의 경우 LangChain은 도구 호출을 사용하여 동일한 결과를 달성합니다. 이는 대부분의 최신 모델인 도구 호출을 지원하는 모든 모델에서 작동합니다.

### Pydantic 모델

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

class ProductReview(BaseModel):
    """Analysis of a product review."""
    rating: int | None = Field(description="The rating of the product", ge=1, le=5)
    sentiment: Literal["positive", "negative"] = Field(description="The sentiment of the review")
    key_points: list[str] = Field(description="The key points of the review. Lowercase, 1-3 words each.")

agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(ProductReview)
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Analyze this review: 'Great product: 5 out of 5 stars. Fast shipping, but expensive'"}]
})

print(result["structured_response"])
# ProductReview(rating=5, sentiment='positive', key_points=['fast shipping', 'expensive'])

### Union 타입

여러 스키마 옵션을 제공할 수 있습니다. 모델은 컨텍스트에 따라 가장 적절한 스키마를 선택합니다.

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, Union
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

class ProductReview(BaseModel):
    """Analysis of a product review."""
    rating: int | None = Field(description="The rating of the product", ge=1, le=5)
    sentiment: Literal["positive", "negative"] = Field(description="The sentiment of the review")
    key_points: list[str] = Field(description="The key points of the review. Lowercase, 1-3 words each.")

class CustomerComplaint(BaseModel):
    """A customer complaint about a product or service."""
    issue_type: Literal["product", "service", "shipping", "billing"] = Field(description="The type of issue")
    severity: Literal["low", "medium", "high"] = Field(description="The severity of the complaint")
    description: str = Field(description="Brief description of the complaint")

agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(Union[ProductReview, CustomerComplaint])
)

# 리뷰 분석
result1 = agent.invoke({
    "messages": [{"role": "user", "content": "Analyze this review: 'Great product: 5 out of 5 stars. Fast shipping, but expensive'"}]
})
print("Review:", result1["structured_response"])

# 불만 처리
result2 = agent.invoke({
    "messages": [{"role": "user", "content": "Customer complaint: Package arrived damaged and contents were broken"}]
})
print("Complaint:", result2["structured_response"])

### 커스텀 도구 메시지 콘텐츠

`tool_message_content` 매개변수를 사용하면 구조화된 출력이 생성될 때 대화 기록에 나타나는 메시지를 커스터마이징할 수 있습니다.

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

class MeetingAction(BaseModel):
    """Action items extracted from a meeting transcript."""
    task: str = Field(description="The specific task to be completed")
    assignee: str = Field(description="Person responsible for the task")
    priority: Literal["low", "medium", "high"] = Field(description="Priority level")

agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
        schema=MeetingAction,
        tool_message_content="Action item captured and added to meeting notes!"
    )
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "From our meeting: Sarah needs to update the project timeline as soon as possible"}]
})

print(result["structured_response"])
# MeetingAction(task='Update the project timeline', assignee='Sarah', priority='high')

## 오류 처리

모델은 도구 호출을 통해 구조화된 출력을 생성할 때 실수를 할 수 있습니다. LangChain은 이러한 오류를 자동으로 처리하는 지능형 재시도 메커니즘을 제공합니다.

### 스키마 검증 오류

구조화된 출력이 예상 스키마와 일치하지 않으면 에이전트는 특정 오류 피드백을 제공합니다.

In [ ]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

class ProductRating(BaseModel):
    rating: int | None = Field(description="Rating from 1-5", ge=1, le=5)
    comment: str = Field(description="Review comment")

agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(ProductRating),  # 기본값: handle_errors=True
    system_prompt="You are a helpful assistant that parses product reviews. Do not make any field or value up."
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Parse this: Amazing product, 10/10!"}]
})

print(result["structured_response"])
# ProductRating(rating=5, comment='Amazing product')
# 모델이 자동으로 수정하여 10을 5로 변경

### 오류 처리 전략

`handle_errors` 매개변수를 사용하여 오류 처리 방법을 커스터마이징할 수 있습니다.

#### 커스텀 오류 메시지

In [ ]:
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
        schema=ProductRating,
        handle_errors="Please provide a valid rating between 1-5 and include a comment."
    )
)

#### 특정 예외만 처리

In [ ]:
agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
        schema=ProductRating,
        handle_errors=ValueError  # ValueError만 재시도, 다른 예외는 발생
    )
)

#### 여러 예외 유형 처리

In [ ]:
agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
        schema=ProductRating,
        handle_errors=(ValueError, TypeError)  # ValueError 및 TypeError 재시도
    )
)

#### 커스텀 오류 핸들러 함수

In [ ]:
from langchain.agents.structured_output import (
    StructuredOutputValidationError,
    MultipleStructuredOutputsError
)

def custom_error_handler(error: Exception) -> str:
    if isinstance(error, StructuredOutputValidationError):
        return "There was an issue with the format. Try again."
    elif isinstance(error, MultipleStructuredOutputsError):
        return "Multiple structured outputs were returned. Pick the most relevant one."
    else:
        return f"Error: {str(error)}"

agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
        schema=ProductRating,
        handle_errors=custom_error_handler
    )
)

#### 오류 처리 없음

In [ ]:
agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
        schema=ProductRating,
        handle_errors=False  # 모든 오류 발생
    )
)

## 종합 예제

복잡한 구조화된 출력을 사용하는 실용적인 예제입니다.

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, Union
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

# 여러 응답 유형 정의
class BookRecommendation(BaseModel):
    """Book recommendation with details."""
    title: str = Field(description="Book title")
    author: str = Field(description="Author name")
    genre: Literal["fiction", "non-fiction", "science", "history", "biography"] = Field(description="Book genre")
    rating: int = Field(description="Rating from 1-5", ge=1, le=5)
    summary: str = Field(description="Brief summary of the book")

class MovieRecommendation(BaseModel):
    """Movie recommendation with details."""
    title: str = Field(description="Movie title")
    director: str = Field(description="Director name")
    year: int = Field(description="Release year")
    genre: Literal["action", "comedy", "drama", "horror", "sci-fi"] = Field(description="Movie genre")
    rating: int = Field(description="Rating from 1-5", ge=1, le=5)

# 에이전트 생성
agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
        schema=Union[BookRecommendation, MovieRecommendation],
        handle_errors=True
    ),
    system_prompt="You are a helpful entertainment recommendation assistant."
)

# 책 추천
result1 = agent.invoke({
    "messages": [{"role": "user", "content": "Recommend a good science fiction book"}]
})
print("Book recommendation:")
print(result1["structured_response"])

# 영화 추천
result2 = agent.invoke({
    "messages": [{"role": "user", "content": "Recommend a comedy movie from the 2000s"}]
})
print("\nMovie recommendation:")
print(result2["structured_response"])